In [ ]:
! pip install faiss-cpu langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.0 MB/s eta 0:00:00


In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4

In [8]:
%%capture --no-stderr
%pip install langchain-openai

In [22]:

!pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.8 MB/s eta 0:00:00


In [25]:
from google.colab import userdata
groq_api_key=userdata.get('groqAPI')

In [4]:
# import getpass
# import os

# os.environ["MISTRAL_API_KEY"] = groq_api_key

# from langchain_mistralai import ChatMistralAI

# llm = ChatMistralAI(model="mistral-large-latest")

In [26]:

import os
os.environ["GROQ_API_KEY"] = groq_api_key

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [15]:
%%capture --no-stderr
%pip install --upgrade --quiet  sentence_transformers

In [12]:
from google.colab import userdata
HF_API_KEY=userdata.get('HF_TOKEN')
os.environ["OPENAI_API_KEY"] = HF_API_KEY


In [16]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=hf)
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [28]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

'Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps.'

In [29]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [31]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Common ways of doing task decomposition include: 1) using a language model (LLM) with simple prompting, 2) using task-specific instructions, and 3) with human inputs.


In [32]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [33]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition is the process of breaking down a complex task into smaller, simpler steps.'

In [36]:
conversational_rag_chain.invoke(
    {"input": "how this technique help us?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Chain of Thought (CoT) is a standard prompting technique that enhances model performance on complex tasks by instructing the model to "think step by step", decomposing hard tasks into smaller and simpler steps.'

In [39]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition is the process of breaking down a complex task into smaller, simpler steps.

User: What are common ways of doing it?

AI: I don't know

User: how this technique help us?

AI: Chain of Thought (CoT) is a standard prompting technique that enhances model performance on complex tasks by instructing the model to "think step by step", decomposing hard tasks into smaller and simpler steps.

User: what is chain of thought?

AI: Chain of Thought (CoT) is a prompting technique that guides the model to generate a step-by-step explanation of its thought process, resulting in more accurate and interpretable outputs.



In [40]:
while True:
    user_input = input("User: ")
    if user_input == "exit":
        break
    conversational_rag_chain.invoke(
        {"input":user_input},
        config={"configurable": {"session_id": "abc123"}},
    )["answer"]
    for message in store["abc123"].messages:
        if isinstance(message, AIMessage):
            prefix = "AI"
        else:
            prefix = "User"

        print(f"{prefix}: {message.content}\n")

User: explain decomposition in a simple language?


User: What is Task Decomposition?

AI: Task decomposition is the process of breaking down a complex task into smaller, simpler steps.

User: What are common ways of doing it?

AI: I don't know

User: how this technique help us?

AI: Chain of Thought (CoT) is a standard prompting technique that enhances model performance on complex tasks by instructing the model to "think step by step", decomposing hard tasks into smaller and simpler steps.

User: what is chain of thought?

AI: Chain of Thought (CoT) is a prompting technique that guides the model to generate a step-by-step explanation of its thought process, resulting in more accurate and interpretable outputs.

User: explain decomposition in a simple language?

AI: Decomposition is like breaking down a big puzzle into smaller, easier-to-solve puzzles.

User: ok fine . but how to proceed with it?


User: What is Task Decomposition?

AI: Task decomposition is the process of breaking down a complex task into smaller, simpler steps.

User: What are common ways of doing it?

AI: I don't know

User: how this technique help us?

AI: Chain of Thought (CoT) is a standard prompting technique that enhances model performance on complex tasks by instructing the model to "think step by step", decomposing hard tasks into smaller and simpler steps.

User: what is chain of thought?

AI: Chain of Thought (CoT) is a prompting technique that guides the model to generate a step-by-step explanation of its thought process, resulting in more accurate and interpretable outputs.

User: explain decomposition in a simple language?

AI: Decomposition is like breaking down a big puzzle into smaller, easier-to-solve puzzles.

User: ok fine . but how to proceed with it?

AI: I don't know

User: please try to find how to proceed with it so that i can start my work?


User: What is Task Decomposition?

AI: Task decomposition is the process of breaking down a complex task into smaller, simpler steps.

User: What are common ways of doing it?

AI: I don't know

User: how this technique help us?

AI: Chain of Thought (CoT) is a standard prompting technique that enhances model performance on complex tasks by instructing the model to "think step by step", decomposing hard tasks into smaller and simpler steps.

User: what is chain of thought?

AI: Chain of Thought (CoT) is a prompting technique that guides the model to generate a step-by-step explanation of its thought process, resulting in more accurate and interpretable outputs.

User: explain decomposition in a simple language?

AI: Decomposition is like breaking down a big puzzle into smaller, easier-to-solve puzzles.

User: ok fine . but how to proceed with it?

AI: I don't know

User: please try to find how to proceed with it so that i can start my work?

AI: According to the text, you can proceed wi